# Label side

In [1]:
import numpy as np
from jesse import helpers, research

_, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-03-20"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

trading_1m.shape

(2743200, 6)

In [2]:
np.save("data/btc_1m.npy", trading_1m)

In [1]:
import numpy as np

trading_1m = np.load("data/btc_1m.npy")
trading_1m.shape

(2743200, 6)

In [2]:
from custom_indicators.toolbox.bar.dollar_bar import estimate_dollar_bar_threshold

for i in [30, 120, 60 * 6]:
    threshold = estimate_dollar_bar_threshold(trading_1m, i)
    print(f"{i}m: {threshold}")

30m: 248550043.9622903
120m: 1052682535.8991117
360m: 3216529968.747286


In [3]:
from jesse import helpers

from custom_indicators.config import DOLLAR_BAR_THRESHOLD_MID
from custom_indicators.toolbox.bar.dollar_bar import build_dollar_bar

tp = helpers.date_to_timestamp("2025-03-01")

label_bar = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_MID)

print(label_bar.shape)

(22854, 6)


In [4]:
from custom_indicators.toolbox.labeler.zigzag_labeler import zigzag

res = zigzag(label_bar[:, 2], 0.03)
np.unique(res, return_counts=True)

(array([-1,  1]), array([10685, 12169]))

In [5]:
label = np.column_stack((label_bar, res))
label

array([[1.57797306e+12, 7.10767000e+03, 7.27437000e+03, ...,
        6.86344000e+03, 1.48997210e+05, 1.00000000e+00],
       [1.57804146e+12, 7.27437000e+03, 7.34161000e+03, ...,
        7.22843000e+03, 1.44005650e+05, 1.00000000e+00],
       [1.57812180e+12, 7.34181000e+03, 7.43621000e+03, ...,
        7.26921000e+03, 1.42629374e+05, 1.00000000e+00],
       ...,
       [1.74241434e+12, 8.52662000e+04, 8.56226000e+04, ...,
        8.51639000e+04, 1.24091370e+04, 1.00000000e+00],
       [1.74242046e+12, 8.56226000e+04, 8.61519000e+04, ...,
        8.54414000e+04, 1.22285510e+04, 1.00000000e+00],
       [1.74242526e+12, 8.61520000e+04, 8.68098000e+04, ...,
        8.61400000e+04, 1.22275090e+04, 1.00000000e+00]])

In [6]:
np.save("data/label_side.npy", label[label[:, 0] <= tp])
print(label.shape)
print(label[label[:, 0] <= tp].shape)

(22854, 7)
(22445, 7)


In [7]:
from jesse import utils

df_dollar_bar = utils.numpy_candles_to_dataframe(
    label_bar[label_bar[:, 0] <= tp], name_date="time"
)
df_dollar_bar["zigzag"] = res[label_bar[:, 0] <= tp]
df_dollar_bar.head(1)

,time,open,close,high,low,volume,zigzag
2020-01-02 13:51:00,2020-01-02 13:51:00,7107.67,7274.37,7300.0,6863.44,148997.21,1


In [8]:
from lightweight_charts import Chart

chart = Chart(inner_width=1, inner_height=0.8)
chart2 = chart.create_subchart(position="bottom", width=1, height=0.2, sync=True)

chart.set(df_dollar_bar[["time", "open", "high", "low", "close", "volume"]])

line = chart2.create_line(name="zigzag", style="dotted")
line.set(df_dollar_bar[["time", "zigzag"]])

chart.show(block=True)

2025-03-24 20:48:56.823 python[72665:7096948] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-24 20:48:56.823 python[72665:7096948] +[IMKInputSession subclass]: chose IMKInputSession_Modern


RuntimeError: asyncio.run() cannot be called from a running event loop

# Label meta

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from custom_indicators.config import SIDE_ALL

label_side_1h = np.load("data/label_side.npy")
print(label_side_1h.shape)
print(np.unique(label_side_1h[:, 6], return_counts=True))

df_features = pd.read_parquet("data/features_2h.parquet")
print(df_features.shape)

side_model = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
side_model_pred_prob = side_model.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_pred_prob
side_model_pred = np.where(side_model_pred_prob > 0.5, 1, -1)
print(np.unique(side_model_pred, return_counts=True))

(22445, 7)
(array([-1.,  1.]), array([10469, 11976]))
(22445, 11841)
(array([-1,  1]), array([10137, 12308]))


In [2]:
meta_label = (side_model_pred.astype(int) == label_side_1h[:, 6].astype(int)).astype(
    int
)
np.unique(meta_label, return_counts=True)

(array([0, 1]), array([ 4394, 18051]))

In [3]:
label_meta_1h = label_side_1h.copy()
label_meta_1h[:, 6] = meta_label
np.save("data/label_meta.npy", label_meta_1h)